# Improve embedding with HuggingFace pre-trained models

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

## 1. Imports

In [7]:
#classic
import pandas as pd
import numpy as np
from datetime import datetime
import pickle


#visualization
import matplotlib.pyplot as plt

#models
from transformers import AutoTokenizer, TFAutoModel

import tensorflow as tf

## 2. Data

,title,plot_synopsis
0,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'..."
1,Scarface,"In May 1980, a Cuban man named Tony Montana (A..."
5,Flightplan,Kyle Pratt (Jodie Foster) is a propulsion engi...
6,Little Caesar,Small-time Italian-American criminals Caesar E...
7,Savages,The movie begins with a video being shot of me...


## 4. Using pre-trained tiny-bert

### 1. Text embedding with bert

In [11]:
model_name = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name, from_pt = True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'bert.embeddings.position_ids', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

### 1.1. Embed the (original) plots

In [1]:
# import data
df = pd.read_pickle('/content/drive/MyDrive/shared_data/raw_data_shared/data_movie.pkl')
df.head()

NameError: name 'pd' is not defined

In [12]:
plot = df['plot_synopsis'].tolist()

# Tokenize the text data
token_tensor = tokenizer(plot, padding='max_length', max_length= 500, truncation=True, return_tensors="tf")

# Create input tensors
input_tensor = token_tensor['input_ids']

# Generate embeddings
prediction = model.predict(input_tensor)

194/194 [==============================] - 262s 1s/step


2024-02-08 14:28:07.459148: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1588224000 exceeds 10% of free system memory.


In [13]:
prediction

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=array([[[-0.53041697,  0.07894692, -4.1747184 , ...,  0.1208056 ,
         -0.26163447,  1.4187179 ],
        [-1.2223986 ,  0.2652639 ,  0.31511763, ..., -2.149884  ,
         -0.3219931 ,  1.6733292 ],
        [-0.84478337,  0.3376569 , -0.7529081 , ..., -2.9411268 ,
          0.18474263,  0.29692876],
        ...,
        [-0.64974487,  0.19274138, -1.119652  , ..., -0.16245715,
         -1.546545  ,  1.5404472 ],
        [-0.05792686, -0.09606716, -0.3784339 , ..., -1.3717613 ,
          0.20261516,  0.20651756],
        [-0.73357034,  0.5554948 ,  0.4314581 , ..., -1.3273001 ,
         -0.7229799 ,  0.76031506]],

       [[-1.9052038 , -0.8651169 , -4.592863  , ..., -0.17127931,
         -1.2534415 ,  1.5046825 ],
        [-1.3443586 , -0.42073253, -0.6326513 , ..., -3.1596131 ,
         -0.0952716 ,  0.46136558],
        [-1.5785536 , -0.05970723, -0.8478688 , ..., -1.8083404 ,
         -0.7600982 ,  1.138109  ],
  

In [14]:
# Process the embeddings as np
plot_embeddings = prediction.last_hidden_state[:, 0, :]

In [ ]:
np.save('/content/drive/MyDrive/shared_data/processed_data_shared/embedding_plot.npy', plot_embeddings)

### 1.2. Embed the (synthetic) summaries

In [ ]:
# import data
df = pd.read_csv('/content/drive/MyDrive/shared_data/raw_data_shared/movie_with_summary.csv')
df.head()

In [ ]:
summaries = df['geny_summary'].tolist()

# Tokenize the text data
token_tensor = tokenizer(summaries, padding='max_length', max_length= 500, truncation=True, return_tensors="tf")

# Create input tensors
input_tensor = token_tensor['input_ids']

# Generate embeddings
prediction = model.predict(input_tensor)

In [ ]:
# Process the embeddings as np
summary_embeddings = prediction.last_hidden_state[:, 0, :]

In [ ]:
np.save('/content/drive/MyDrive/shared_data/processed_data_shared/embedding_summary.npy', summary_embeddings)

**Movie similarity with cosinus similarity**

In [15]:
# from sklearn.metrics.pairwise import cosine_similarity

# # Calculate cosine similarity between embeddings
# similarity_matrix = cosine_similarity(embeddings)

# # Example: Get top 10 most similar movies for each movie
# num_movies = similarity_matrix.shape[0]
# top_n = 1  # Number of similar movies to retrieve

# for i in range(num_movies):
#     # Sort similarity scores for movie i
#     sim_scores = sorted(enumerate(similarity_matrix[i]), key=lambda x: x[1], reverse=True)
#     # Exclude movie i itself
#     sim_scores = sim_scores[1:]
#     # Get indices of top similar movies
#     top_indices = [idx for idx, _ in sim_scores[:top_n]]
#     # Print movie titles of top similar movies
#     print(f"Top {top_n} similar movies for '{movie_titles[i]}':")
#     for idx in top_indices:
#         print(f"  - {movie_titles[idx]}")
#     print()

**Movie recommendation with cosine similarity**

In [16]:
# # Calculate Movie Similarity

# similarity_matrix = cosine_similarity(embeddings)

# # Select User Preferences
# user_input = input("Enter a movie title you like: ")

# # Retrieve Similar Movies
# movie_idx = movie_titles.index(user_input)
# sim_scores = list(enumerate(similarity_matrix[movie_idx]))
# sim_scores_sorted = sorted(sim_scores, key=lambda x: x[1], reverse=True)

# # Filter Recommendations (optional)
# top_n = 5  # Number of recommendations to display
# recommended_movies = [movie_titles[idx] for idx, _ in sim_scores_sorted[1:top_n+1]]  # Exclude the input movie itself

# # Display Recommendations
# print(f"Recommended movies based on '{user_input}':")
# for movie in recommended_movies:
#     print(movie)